In [1]:
#from whisper_jax import FlaxWhisperForConditionalGeneration
from transformers import WhisperForConditionalGeneration
import torch
from functions import *
from functools import partial

import evaluate
from transformers import AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

IsTPU = False
batch_size = 16
device = 'cuda'
dtype = torch.float32
speech_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/dlsprint/validation_files/'
data_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/dlsprint/validation.csv'
add_audio2 = '/home/zhenlan/Desktop/Projects/Bengali ASR/RESPIN/dev/'
model_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/model_all3'
speech_path0 = 'data/train_mp3s/'
data_path0 = 'data/train.csv'
num_workers = 8
output_path = ''


In [2]:
pad_to_multiple_of = 1
max_length_gen = 24
epochs = 1
verbose = 5
learning_rate=4e-4
clip = 1e-2
# tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v2", language="bn", task="transcribe")
#tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5")
tokenizer.bos_token = tokenizer.bos_token_id = None
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v2")
text = pd.read_csv(data_path)
text0 = pd.read_csv(data_path0)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


In [3]:
df = pd.read_csv('text',names=['code'])
df[['code','sentence']] = df["code"].str.split(" ", n=1, expand=True)
df.sentence = df.sentence.str.strip()

In [4]:
dataset0 = AudioDataset(text0,speech_path0,lambda x:x.id+'.mp3',\
                        orig_sr=32000, target_sr=16000)
train_loader_CE0 = DataLoader(dataset0, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
                                           pad_to_multiple_of=pad_to_multiple_of,IsTrain=True,IsTPU=IsTPU,batch_size=batch_size))
train_loader_WER0 = DataLoader(dataset0, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
                                           pad_to_multiple_of=pad_to_multiple_of,IsTrain=False,IsTPU=IsTPU,batch_size=batch_size))

dataset1 = AudioDataset(text,speech_path,lambda x:x.path,\
                       None,orig_sr=32000, target_sr=16000)
train_loader_CE = DataLoader(dataset1, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
                                           pad_to_multiple_of=pad_to_multiple_of,IsTrain=True,IsTPU=IsTPU,batch_size=batch_size))
train_loader_WER = DataLoader(dataset1, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
                                           pad_to_multiple_of=pad_to_multiple_of,IsTrain=False,IsTPU=IsTPU,batch_size=batch_size))

dataset2 = AudioDataset(df,add_audio2,\
                        lambda x:x.code.split('_')[-1] + '.wav',orig_sr=16000, target_sr=16000)
train_loader_CE2 = DataLoader(dataset2, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
                                           pad_to_multiple_of=pad_to_multiple_of,IsTrain=True,IsTPU=IsTPU,batch_size=batch_size))
train_loader_WER2 = DataLoader(dataset2, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
                                           pad_to_multiple_of=pad_to_multiple_of,IsTrain=False,IsTPU=IsTPU,batch_size=batch_size))

In [5]:
model = WhisperForConditionalGeneration.from_pretrained(model_path)
model.eval()
model.config.forced_decoder_ids = None
model.config.bos_token_id = None
model.config.suppress_tokens = None
model.config.decoder_start_token_id = None
model.generation_config.decoder_start_token_id = [50258, 50302, 50359, 50363]# '<|startoftranscript|><|bn|><|transcribe|><|notimestamps|>
model.generation_config.forced_decoder_ids = None
"""A list of pairs of integers which indicates a mapping from generation indices to token indices 
that will be forced before sampling. For example, [[0, 123]] means the first generated token 
will always be a token of index 123."""
model.generation_config.suppress_tokens = None
model.generation_config.begin_suppress_tokens = None
model.generation_config.bos_token_id = None
model = model.to(device)

In [6]:
metric = evaluate.load("wer")
def metric_one_step(audio,txt):
    with torch.no_grad():
        # generated_ids = model.generate(torch.tensor(audio,device=device),max_length=max_length_gen, num_beams=1, do_sample=False)
        generated_ids = model.generate(torch.tensor(audio,device=device),\
                               max_length=max_length_gen,num_beams=5,\
                               early_stopping=True, no_repeat_ngram_size=2,\
                               min_new_tokens=2, eos_token_id=50257,do_sample=False)
    transcriptions = tokenizer.batch_decode(generated_ids.tolist(), skip_special_tokens=True)
    wer = metric.compute(predictions=transcriptions, references=txt)
    return wer

def eval_one_step(audio,input_ids,attention_mask):
    audio,input_ids,attention_mask = torch.tensor(audio,dtype=dtype,device=device),\
                                 torch.tensor(input_ids,dtype=torch.long,device=device),\
                                 torch.tensor(attention_mask,dtype=torch.long,device=device)
    with torch.no_grad():
        logits = model(audio,decoder_input_ids=input_ids,decoder_attention_mask=attention_mask).logits
        l = torch.mean(torch.nn.CrossEntropyLoss(reduction='none')(logits[:,3:-1].reshape(-1,51865),input_ids[:,4:].reshape(-1))*attention_mask[:,4:].reshape(-1))
        return l

DATASET0

In [7]:
import logging
logging.basicConfig(filename=model_path+'/'+'performance.log', filemode='w', level=logging.INFO,
                    format='%(message)s', datefmt='%Y-%m-%d %H:%M:%S')

In [8]:
%%time
# # model_all2
test_loss = 0
for j,(audio,input_ids,attention_mask) in enumerate(train_loader_CE0):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = eval_one_step(audio,input_ids,attention_mask)
    test_loss += l.item()
    if j>20:
        break
test_loss /= (j+1)
logging.info(f"CE loss on data0: {test_loss:.4f}")

CPU times: user 1.23 s, sys: 356 ms, total: 1.59 s
Wall time: 2.31 s


In [9]:
%%time
# model_all2
test_loss = 0
for j,(audio,txt) in enumerate(train_loader_WER0):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = metric_one_step(audio,txt)
    test_loss += l
    if j>20:
        break
test_loss /= (j+1)
logging.info(f"WER loss on data0: {test_loss:.4f}")

CPU times: user 6.68 s, sys: 211 ms, total: 6.89 s
Wall time: 7.51 s


DATASET1

In [10]:
%%time
# # model_all2
test_loss = 0
for j,(audio,input_ids,attention_mask) in enumerate(train_loader_CE):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = eval_one_step(audio,input_ids,attention_mask)
    test_loss += l.item()
    if j>20:
        break
test_loss /= (j+1)
logging.info(f"CE loss on data1: {test_loss:.4f}")

CPU times: user 1.19 s, sys: 215 ms, total: 1.4 s
Wall time: 2.28 s


In [11]:
%%time
# model_all2
test_loss = 0
for j,(audio,txt) in enumerate(train_loader_WER):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = metric_one_step(audio,txt)
    test_loss += l
    if j>20:
        break
test_loss /= (j+1)
logging.info(f"WER loss on data1: {test_loss:.4f}")

CPU times: user 7.21 s, sys: 184 ms, total: 7.39 s
Wall time: 8.25 s


DATASET2

In [12]:
%%time
# # model_all2
test_loss = 0
for j,(audio,input_ids,attention_mask) in enumerate(train_loader_CE2):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = eval_one_step(audio,input_ids,attention_mask)
    test_loss += l.item()
    if j>20:
        break
test_loss /= (j+1)
logging.info(f"CE loss on data2: {test_loss:.4f}")

CPU times: user 2.22 s, sys: 562 ms, total: 2.78 s
Wall time: 3.62 s


In [13]:
%%time
# model_all2
test_loss = 0
for j,(audio,txt) in enumerate(train_loader_WER2):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = metric_one_step(audio,txt)
    test_loss += l
    if j>20:
        break
test_loss /= (j+1)
logging.info(f"WER loss on data2: {test_loss:.4f}")

CPU times: user 10.4 s, sys: 559 ms, total: 11 s
Wall time: 11.8 s
